In [23]:
import vk_api
import time
from progressbar import ProgressBar, Percentage, Bar, ETA
from itertools import groupby

In [24]:
API_VERSION = '5.92'
APP_ID = 6973103

# 2FA обработчик
def auth_handler():
    key = input('Введите код двухфакторной аутентификации: ')
    remember_device = True
    return key, remember_device

# captcha обработчик
def captcha_handler(captcha):
    key = input(f"Введите капчу ({captcha.get_url()}): ").strip()
    return captcha.try_again(key)

# авторизация по токену или логин-паролю
def login_vk(token=None, login=None, password=None):
    try:
        if token:
            vk_session = vk_api.VkApi(token=token, app_id=APP_ID, auth_handler=auth_handler,
                                      api_version=API_VERSION)
        elif login and password:
            vk_session = vk_api.VkApi(login, password, captcha_handler=captcha_handler, app_id=APP_ID,
                                      api_version=API_VERSION, auth_handler=auth_handler)
            vk_session.auth(token_only=True, reauth=True)
        else:
            raise KeyError('Не введен токен или пара логин-пароль')
            
        return vk_session.get_api()
    except Exception as e:
        print(e)

In [25]:
token = '82ae80bf32f3ed5e1b363a287516063a4440ef2cbd14de516314b3c590dc3ea75ffe2574ee9e90ea5eecf'
vk = login_vk(token)

In [26]:
def get_user_id(vk):
    # получить свой id
    try:
        user_id = vk.users.get()[0]['id']
        if not user_id:
            raise KeyError('Не удалось получить данные пользователя')
        return user_id
    except Exception as e:
        print(e)


def get_friends(vk, user_id = get_user_id(vk)):
    try:
        friends = vk.friends.get(user_id = user_id, fields='photo_200_orig')
        return friends['items']
    except:
        return [{'id': None,
              'first_name': None,
              'last_name': None,
              'is_closed': None,
              'can_access_closed': None,
              'photo_200_orig': None,
              'online': None}]

    
def delete_repeating(l):
    n = []
    for i in l:
        if i not in n:
            n.append(i)
    return n


def get_friends_of_friends(vk):
    
    
    friends = get_friends(vk)
    
    # визуалзация проресса
    progress, progress_maxval = 0, len(friends)
    pbar = ProgressBar(widgets=['Progress ', Percentage(), Bar(), ' ', ETA(), ],
                   maxval=progress_maxval).start()
    
    
    friends_temp = [d['id'] for d in friends if 'id' in d]
    for friend in friends_temp:
        friends.extend(get_friends(vk, friend))
        
        progress += 1
        pbar.update(progress)

    friends_non_repeating = delete_repeating(friends)
    
    pbar.finish()
    return friends_non_repeating
    

In [27]:
start_time = time.time()

z = get_friends_of_friends(vk)

print("--- %s seconds ---" % (time.time() - start_time))

--- 141.47579979896545 seconds ---


In [36]:
# f = get_friends(vk)

# ids = [d['id'] for d in f if 'id' in d]
# str(ids)

z[1000]

{'id': 10858813,
 'first_name': 'Александр',
 'last_name': 'Кожанов',
 'is_closed': False,
 'can_access_closed': True,
 'photo_200_orig': 'https://vk.com/images/camera_200.png?ava=1',
 'online': 0}

In [31]:
import requests

def save_photo(url):
    temp = requests.get(url)
    return temp

a = save_photo('https://pp.userapi.com/c850420/v850420767/5571d/Fa_DhDh6R3I.jpg?ava=1')

31399

In [ ]:
async def asnc(link, title):
    async with aiohttp.ClientSession() as session:
        async with session.get('https://stackoverflow.com' + link) as resp:
            fin = asparse(await resp.text())
            return title, fin


def parsing(link):
    reqdata = {}
    page = requests.get(link)
    if page.ok is True:
        soup = BeautifulSoup(page.content, 'html.parser')
        themes = soup.find_all('a', attrs={'class': 'question-hyperlink'})
        tasks = []
        loop = asyncio.new_event_loop()
        for theme in themes:
            tasks.append(loop.create_task(asnc(theme['href'], theme.text)))
        fin = loop.run_until_complete(asyncio.wait(tasks))
        loop.close()
        for i in fin[0]:
            if i._result is not None:
                reqdata[i._result[0]] = i._result[1]
    return reqdata

In [25]:
import shutil
from itertools import repeat
from multiprocessing.pool import Pool
from os import cpu_count
from os.path import join, exists
PULL_PROCESSES = 4 * cpu_count()

def download_engine(object_list, path):
    with Pool(PULL_PROCESSES) as pool:
        pool.starmap(download, zip(object_list, repeat(path)))

def download(obj, root):
    try:
        filename, url = get_filename_url(obj)
        path = join(root, filename)
        if not exists(path):
            r = requests.get(url, stream=True, timeout=(30, 5))
            with open(path, 'wb') as f:
                shutil.copyfileobj(r.raw, f)
    except Exception as e:
        print(e)
        return False

    
def dump_docs(docs, path):
    makedirs(path, exist_ok=True)
    download_engine(docs['docs'], path)

In [15]:
token = "90f00b0911d5c1e849af49a22595725b3b1d1c14235e47542c59687b24ec177a21cae7b56eeef3c52b935"
vk = vk_api.VkApi(token=token, app_id=APP_ID).get_api()


In [16]:
def get_griends_bot(vk, user_id):
    friends = vk.users.get(user_id = user_id, fields = 'photo_200')
    return friends

In [17]:
f = get_griends_bot(vk, 49853293)
f

[{'id': 49853293,
  'first_name': 'Марк',
  'last_name': 'Антипин',
  'is_closed': False,
  'can_access_closed': True,
  'photo_200': 'https://pp.userapi.com/c630222/v630222293/26501/7ddNCcIXCLY.jpg?ava=1'}]

In [20]:
type(str(f[0]['id']))

str

In [21]:
str(f[0]['id'])

'49853293'